In [2]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,KQ__Ex,KQ__Fa,KQ__Gd
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,2,2008,WD,Normal,208500,0,0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,5,2007,WD,Normal,181500,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,0,9,2008,WD,Normal,223500,0,0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,0,2,2006,WD,Abnorml,140000,0,0,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,0,12,2008,WD,Normal,250000,0,0,1
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,Shed,700,10,2009,WD,Normal,143000,0,0,0
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,8,2007,WD,Normal,307000,0,0,1
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,Shed,350,11,2009,WD,Normal,200000,0,0,0
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,4,2008,WD,Abnorml,129900,0,0,0
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,1,2008,WD,Normal,118000,0,0,0


In [18]:
train = pd.read_csv("../jen/house-prices-advanced-regression-techniques/train.csv")
train.columns
train.set_index("Id", inplace=True)

train = train.loc[train['GrLivArea'] < 4500] # outlier removal

train['FullBath'] = train['FullBath'] + train['BsmtFullBath']
train['HalfBath'] = train['HalfBath'] + train['BsmtHalfBath']
train['Total_porch_sf'] = (train['OpenPorchSF'] + train['3SsnPorch'] +train['EnclosedPorch'] + train['ScreenPorch'] + train['WoodDeckSF'])

train.drop(['FireplaceQu', 'Street', 'Utilities', 'LandContour', 'MasVnrType',
	'Condition2', 'PoolArea', 'LotFrontage', 'Functional',
	'LotConfig', 'Fence', 'BldgType', 'Street',
	'Alley', 'KitchenAbvGr', 'BsmtFinType2', 'Heating',
	'PavedDrive', 'LandContour', 'Condition1', 'GarageCond', 'ExterCond',
	'MSZoning', 'MiscFeature', 'SaleCondition', 'BsmtFinSF2', 'SaleType',
	'BsmtCond', 'MiscVal', 'GarageQual','EnclosedPorch','3SsnPorch',
	'RoofMatl', 'ScreenPorch', 'Condition2', 'PoolQC', 
	'PoolArea', 'BsmtHalfBath', 'BsmtFullBath', 'BsmtQual', 'BsmtExposure',
	'BsmtFinType1', 'BsmtFinSF1', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF'], 
	axis=1, inplace=True)


train.drop(['MSSubClass', 'LotShape', 'Neighborhood', 'YearRemodAdd',
    'Exterior1st', 'Exterior2nd', 'GarageType', 'GarageYrBlt', 'Foundation',
	'GarageFinish', 'GarageCars','OpenPorchSF','WoodDeckSF'], axis=1, inplace=True)


train['MasVnrArea'] = train['MasVnrArea'].fillna(0)
train['TotSF'] = train['GrLivArea']-train['LowQualFinSF'] + train['TotalBsmtSF']
train.drop(['GrLivArea', 'LowQualFinSF', 'TotalBsmtSF'], axis=1, inplace=True)
train.to_csv('../train_clean.csv', index=False)



#Dummify
# HS_dummy = pd.get_dummies(train['HouseStyle'], prefix='HS', prefix_sep='__', drop_first = True)
# train = pd.concat([train.drop('HouseStyle', axis=1), HS_dummy], axis=1)
# EQ_dummy = pd.get_dummies(train['ExterQual'], prefix='EQ', prefix_sep='__', drop_first = True)
# train = pd.concat([train.drop('ExterQual', axis=1), EQ_dummy], axis=1)
# HQC_dummy = pd.get_dummies(train['HeatingQC'], prefix='HQC', prefix_sep='__',drop_first = True)
# train = pd.concat([train.drop('HeatingQC', axis=1), HQC_dummy], axis=1)
# KQ_dummy = pd.get_dummies(train['KitchenQual'], prefix='KQ', prefix_sep='__', drop_first = True)
# train = pd.concat([train.drop('KitchenQual', axis=1), KQ_dummy], axis=1)
# ## 
print(train.columns)

Index(['LotArea', 'LandSlope', 'HouseStyle', 'OverallQual', 'OverallCond',
       'YearBuilt', 'RoofStyle', 'MasVnrArea', 'ExterQual', 'HeatingQC',
       'CentralAir', 'Electrical', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces', 'GarageArea', 'MoSold',
       'YrSold', 'SalePrice', 'Total_porch_sf', 'TotSF'],
      dtype='object')


In [19]:
train.dtypes

LotArea             int64
LandSlope          object
HouseStyle         object
OverallQual         int64
OverallCond         int64
YearBuilt           int64
RoofStyle          object
MasVnrArea        float64
ExterQual          object
HeatingQC          object
CentralAir         object
Electrical         object
FullBath            int64
HalfBath            int64
BedroomAbvGr        int64
KitchenQual        object
TotRmsAbvGrd        int64
Fireplaces          int64
GarageArea          int64
MoSold              int64
YrSold              int64
SalePrice           int64
Total_porch_sf      int64
TotSF               int64
dtype: object

In [20]:
#train = train[['OverallQual','OverallCond', 'YearBuilt','SalePrice', 'KQ__Ex']]

In [21]:
#train = train[train['KQ__Ex']==1]
#train = train.drop('KQ__Ex', axis=1)

In [22]:
#train.sort_values('SalePrice')
train['Qual/Cond'] = train['OverallQual']/train['OverallCond']

In [23]:
#train.sort_values('Qual/Cond',ascending=False)
train.to_csv("undervalued_houses.csv",index=False)